# Creating and Managing Messages
An Assistant represents an entity that can be configured to respond to a user's messages using several parameters like model, instructions, and tools. 

A Thread represents a conversation between a user and one or many Assistants. You can create a Thread when a user (or your AI application) starts a conversation with your Assistant.

The contents of the messages your users or applications create are added as Message objects to the Thread. Messages can contain both text and files. There is no limit to the number of Messages you can add to Threads — we smartly truncate any context that does not fit into the model's context window.

## Creating an Assistant and a Thread Review
Let's create an Assistant and Thread to get us set up for success with our Messages. 

We begin with an Assistant. 

In [4]:
from openai import OpenAI

# Create an instance of the OpenAI class
# This assumes you have the OPENAI_API_KEY environment variable set
client = OpenAI()

# Create an assistant that uses code interpreter.
assistant = client.beta.assistants.create(
    model="gpt-4-turbo",
    instructions="You are a helpful assistant.",
    name="Message Holder",
    metadata={
        "holds_messages": "True",
        "likes_messages": "True",
    },
    temperature=1,
    top_p=1,
)

# Print the details of the created assistant to check the properties.
print(assistant)
print("\n\n")
print(assistant.name)
print(assistant.metadata)

Assistant(id='asst_sfNDj5MsHypnXwrpQkV2gR2F', created_at=1715466848, description=None, instructions='You are a helpful assistant.', metadata={'holds_messages': 'True', 'likes_messages': 'True'}, model='gpt-4-turbo', name='Message Holder', object='assistant', tools=[], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0)



Message Holder
{'holds_messages': 'True', 'likes_messages': 'True'}


Now we create an empty thread that will hold our messages. Remember to hang on to that thread ID.

In [13]:
from openai import OpenAI

# Initialize a client instance of the OpenAI API
client = OpenAI()

# Create a thread using the OpenAI API and store it in a variable
# The metadata specifies a user identifier
thread_holding_messages = client.beta.threads.create(
    metadata={
        "user": "abc123"
    }
)

# Output the result of the thread creation to the console
print(thread_holding_messages)


Thread(id='thread_Sm0jLjJVquSIk4kdMg2hdJPl', created_at=1715467151, metadata={'user': 'abc123'}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


## Creating Messages
To create a message we need to determine which thread to add the message to, the role for the message, and the actual message itself. 

### Simple Message
Here is a simple message with some metadata.

In [40]:
# Create a message in a specific thread using the client's message creation method.
message = client.beta.threads.messages.create(
    thread_id=thread_holding_messages.id,  # ID of the thread where the message will be posted
    role="user",  # Role of the entity posting the message
    content="What is a penguin?",  # The textual content of the message
    metadata={"key": "value"}  # Additional data associated with the message in key-value pairs
)

# Print the entire message object to view its details.
print(message)

# Print a blank line for better readability of the output.
print("\n")

# Print specific attributes of the message.
print(message.id)  # The unique identifier of the message
print(message.content)  # The content of the message
print(message.content[0].text.value)  # Assuming 'content' is a list of text objects, print the value of the first one
print(message.role)  # The role associated with the message


Message(id='msg_LdoGTZudKp1EwqyECL7L0wYF', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What is a penguin?'), type='text')], created_at=1715470869, incomplete_at=None, incomplete_details=None, metadata={'key': 'value'}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_Sm0jLjJVquSIk4kdMg2hdJPl')


msg_LdoGTZudKp1EwqyECL7L0wYF
[TextContentBlock(text=Text(annotations=[], value='What is a penguin?'), type='text')]
What is a penguin?
user


### Message with Vision
Now let's create a more complex message that passes in images.

In [41]:
# Create a file in the system by uploading an image, for vision processing purposes.
vision_file = client.files.create(
    file=open("./artifacts/PuppyDog.jpg", "rb"),  # Opens the image file in binary read mode
    purpose="vision"  # Indicates that the file's purpose is for vision-related processing
)

# Create a message that includes both text and images, asking about differences between them.
message_vision = client.beta.threads.messages.create(
    thread_id=thread_holding_messages.id,  # ID of the thread to post the message in
    role="user",  # Role of the poster (e.g., 'user')
    content=[  # Content list that includes text and images
        {
            "type": "text",
            "text": "What is the difference between these images?"
        },
        {
            "type": "image_url",
            "image_url": {"url": "https://en.wikipedia.org/wiki/File:Cat_August_2010-4.jpg"}
        },
        {
            "type": "image_file",
            "image_file": {"file_id": vision_file.id}  # References the uploaded file by its ID
        },
    ],
)

# Print the message object to output its details.
print(message_vision)


Message(id='msg_j1vyzAxG3HJwqmyEFT2kkHMz', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What is the difference between these images?'), type='text'), ImageFileContentBlock(image_file=None, type='image_url', image_url={'url': 'https://en.wikipedia.org/wiki/File:Cat_August_2010-4.jpg', 'detail': 'auto'}), ImageFileContentBlock(image_file=ImageFile(file_id='file-ZpXKj2U1pGZID1p6uIdDQha1', detail='auto'), type='image_file')], created_at=1715470903, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_Sm0jLjJVquSIk4kdMg2hdJPl')


### Message for Tool Use
Finally, let's pass in a file for Code Interpreter to use.

In [44]:
# Create a file object by uploading a data file for use with assistant tools.
data_file = client.files.create(
    file=open("./artifacts/penguins_size.csv", "rb"),  # Open the CSV file in binary read mode
    purpose="assistants"  # Specify the purpose of the file for assistant tool use
)

# Create a message that instructs to create visualizations from the attached data file.
message_code_interpreter = client.beta.threads.messages.create(
    thread_id=thread_holding_messages.id,  # ID of the thread where the message will be posted
    role="user",  # Role of the entity posting the message
    content="Create three visualizations based on the data in this file.",  # Content detailing what needs to be done with the data
    attachments=[  # Attachments that include the file and tools for processing
        {
            "file_id": data_file.id,  # The ID of the uploaded data file
            "tools": [{"type": "code_interpreter"}]  # Tools specified to interpret the code
        }
    ]
)

# Print the message object to output its details.
print(message_code_interpreter)


Message(id='msg_cwBu1KHuocLtvnCxadmaRSnB', assistant_id=None, attachments=[Attachment(file_id='file-x0aEKXRsLqm1Niw5eCuFdO4M', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Create three visualizations based on the data in this file.'), type='text')], created_at=1715471138, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_Sm0jLjJVquSIk4kdMg2hdJPl')


## Listing Messages
Let's see what messages we have in our thread at this point.

In [47]:
from openai import OpenAI

# Initialize the client using the OpenAI library.
client = OpenAI()

# Retrieve a list of messages from a specific thread using its ID.
thread_messages = client.beta.threads.messages.list(thread_holding_messages.id)
print(thread_messages.data)  # Print the raw data of messages
print("\n")  # Print a newline for better readability of output

# Initialize a counter to keep track of the number of messages.
message_count = 0

# Iterate over each message in the list, printing details and counting them.
for message in thread_messages.data:
    print(message.id)  # Print the unique identifier of each message
    print(message.role)  # Print the role of the message sender
    print(message.content)  # Print the content of the message
    print("\n")  # Print a newline for separation between messages
    message_count += 1  # Increment the message count

# Print the total number of messages processed.
print(f"Total number of messages: {message_count}")


[Message(id='msg_cwBu1KHuocLtvnCxadmaRSnB', assistant_id=None, attachments=[Attachment(file_id='file-x0aEKXRsLqm1Niw5eCuFdO4M', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Create three visualizations based on the data in this file.'), type='text')], created_at=1715471138, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_Sm0jLjJVquSIk4kdMg2hdJPl'), Message(id='msg_KQbiPs4tpex7Vdu8y87OkXmK', assistant_id=None, attachments=[Attachment(file_id='file-OOjsJ3J29zNNr1AXChuTsJ1j', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Create three visualizations based on the data in this file.'), type='text')], created_at=1715471048, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None

## Retrieving a Message
We can always fetch a message if we need to from the thread.

In [39]:
from openai import OpenAI

# Initialize the OpenAI client.
client = OpenAI()

# Retrieve a specific message by its ID from a given thread.
fetched_message = client.beta.threads.messages.retrieve(
    message_id=message.id,  # ID of the message to retrieve
    thread_id=thread_holding_messages.id,  # ID of the thread containing the message
)

# Print the retrieved message object to see its details.
print(fetched_message)



Message(id='msg_9c1YiVpTICD9Rlgjga59ScRJ', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What is a penguin?'), type='text')], created_at=1715467169, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_Sm0jLjJVquSIk4kdMg2hdJPl')


## Modify Messsages
Making changes to messages is easy. Just call the update method.

In [50]:
from openai import OpenAI

# Initialize the OpenAI client.
client = OpenAI()

# Update a specific message by its ID within a given thread.
modified_message = client.beta.threads.messages.update(
    message_id=message.id,  # ID of the message to update
    thread_id=thread_holding_messages.id,  # ID of the thread containing the message
    metadata={  # Metadata to add or update for the message
        "modified": "true",
        "user": "abc123",
    },
)

# Print the modified message object to view its updated details.
print(modified_message)



Message(id='msg_sUsIZ9YCrVSMSpIWPeT2cA9d', assistant_id=None, attachments=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What is a penguin?'), type='text')], created_at=1715467244, incomplete_at=None, incomplete_details=None, metadata={'modified': 'true', 'user': 'abc123'}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_Sm0jLjJVquSIk4kdMg2hdJPl', file_ids=[])


## Delete Messages
Finally, let's clean up after ourselves and delete any messages we don't need any more.

In [ ]:
from openai import OpenAI

# Initialize the OpenAI client.
client = OpenAI()

# Delete a specific message by its ID within a given thread.
deleted_message = client.beta.threads.messages.delete(
    message_id=message.id,  # ID of the message to be deleted
    thread_id=thread_holding_messages.id,  # ID of the thread containing the message
)

# Print the result of the delete operation to verify its success.
print(deleted_message)
